# Null Point Finder

[Null Point Finder]: ../../api_static/plasmapy.analysis.nullpoint.rst

The [null point finder] is functionality that is designed to find and analyze 3D magnetic null point locations and structures using a trilinear interpolation method as described in [Haynes et al. (2007)](https://aip.scitation.org/doi/abs/10.1063/1.2756751).

In this notebook we will briefly cover how the [null point finder] utilizes the trilinear method in order to locate and classify the structures of magnetic null points. 

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from pathlib import Path

from plasmapy.analysis import nullpoint

plt.rcParams["figure.figsize"] = [10.5, 0.56 * 10.5]

## Contents:

1. [How Null Point Finder works](#How-Null-Point-Finder-works)
    1. [Locating a null point](#Locating-a-null-point)
    1. [Classifying a null point](#Classifying-a-null-point)
1. [Running through examples](#Running_through-examples)
    1. [Uniform regular grid with a model function](#Uniform-regular-grid-with-a-model-function)
    1. [Arbitrary Regular Grid](#Arbitrary-regular-grid)

## How Null Point Finder works
Null point finder provides two functions that can locate and classify null points in a magnetic field. The first function is [uniform_null_point_find()](../../api/plasmapy.analysis.nullpoint.uniform_null_point_find.rst)  and the second one is [null_point_find()](../../api/plasmapy.analysis.nullpoint.null_point_find.rst). As the names suggest, `uniform_null_point_find()` is used to locate and classify the magnetic null points of magnetic field located within a regular grid with uniform spacing in each dimension. It requires the user to provide the spacing between grid points in each of the three dimensions in addition to the minimum and maximum coordinate in each dimension. Moreover, it requires the user to provide a function which generates the vector values at a given point. `uniform_null_point_find()` is useful for when the user knows of such a function that accurately models the magnetic field vector at each point in space. 
On the other hand, `null_point_find()` is used when the user does not have an adequate modeling function. It also does not require the grid to have uniform spacing in each dimension. Instead, it will ask the user for three arrays (one for each dimension) of coordinates that determines the desired custom spacing in each of the three dimensions, and then constructs the resulting grid on its own. Furthermore, it requires the user to input all of the three components of magnetic field values, each as a 3D array with the same size as the grid, with each entry representing the strength of that component of the magnetic field for the corresponding grid coordinate. 
Finally, both functions take in as arguments two convergence thresholds that we will discuss later, for locating the null points. 

### Locating a null point
Locating a null point is done via the trilinear analysis method discussed in the paper by [Haynes et al. (2007)](https://aip.scitation.org/doi/abs/10.1063/1.2756751). There are three steps that goes into locating the null points of a given regular grid. 

1. Reduction: First, every grid cell is checked for a simple condition so that we can rule out cells that cannot contain a null point.
2. Trilinear Analysis: Assuming a trilinear field, the cells that have passed the reduction check are then analyzed, so that that we can be sure if they do contain a null point. 
3. Locating the null point: The cell that contains a null point is isolated, and the location of the null point is estimated using the iterative Newton-Raphson method with an initial random guess.

### Classifying a null point
Classification is done by analyzing the Jacobian matrix calculated at the location of null point. The full method is explained in the paper by [Parnell et al. (1996)](https://aip.scitation.org/doi/10.1063/1.871810).

## Running through examples
We will now run through an example for each of the two null point finding functions to see how to properly utilize the null point finder. 

### Uniform regular grid with a model function
First, let's define our modeling function for the magnetic field. 

In [ ]:
def magnetic_field(x, y, z):
    return [(y - 1.5) * (z - 1.5), (x - 1.5) * (z - 1.5), (x - 1.5) * (y - 1.5)]

The vector field defined above has a total of eight null points, located at $(\pm 1.5, \pm 1.5, \pm 1.5)$. Now we will use `uniform_null_point_find()` to locate the null point with all positive components. 

In [ ]:
nullpoint_args = {
    "x_range": [1, 2],
    "y_range": [1, 2],
    "z_range": [1, 2],
    "precision": [0.03, 0.03, 0.03],
    "func": magnetic_field,
}
npoints = nullpoint.uniform_null_point_find(**nullpoint_args)
print(npoints[0].loc)
print(npoints[0].classification)

As we can see `uniform_null_point_find()` correctly identifies the location of the null point in addition to its type, which is a proper radial null.

### Arbitrary Regular Grid
Now we will run through an example where the field components have to be directly provided by the user since an adequate modeling function is not given. 

In [ ]:
nullpoint2_args = {
    "x_arr": [5, 6],
    "y_arr": [5, 6],
    "z_arr": [5, 6],
    "u_arr": np.array([[[-0.5, -0.5], [0.5, 0.5]], [[-0.5, -0.5], [0.5, 0.5]]]),
    "v_arr": np.array([[[-0.5, 0.5], [-0.5, 0.5]], [[-0.5, 0.5], [-0.5, 0.5]]]),
    "w_arr": np.array([[[-0.5, -0.5], [-0.5, -0.5]], [[0.5, 0.5], [0.5, 0.5]]]),
}
npoints2 = nullpoint.null_point_find(**nullpoint2_args)
print(npoints2[0].loc)
print(npoints2[0].classification)

As we can see the magnetic field provided above has a spiral null point located at $(5.5,5.5,5.5)$